In [ ]:
import os, sys, math, time, pickle, gc

import numpy as np

import pandas as pd

from tqdm import tqdm

import torch

import torch.nn as nn

import torch.optim as optim

from torch.utils.data import Dataset, DataLoader


class WideProteinMLP(nn.Module):
    def __init__(self,
                 input_dim,
                 num_classes,
                 hidden_dims=[2048, 4096],
                 dropout=0.3):

        super().__init__()

        layers = []

        layers.append(nn.LayerNorm(input_dim))

        prev = input_dim

        for h in hidden_dims:

            layers.append(nn.Linear(prev, h))

            layers.append(nn.GELU()) 

            layers.append(nn.Dropout(dropout))

            prev = h

        layers.append(nn.Linear(prev, num_classes))

        self.net = nn.Sequential(*layers)

    def forward(self, x):

        return self.net(x)


# ============================================================================

# 1. CONFIG

# ============================================================================

CONFIG = {

    # --- Input Embeddings ---
    "EMBED_DIR": "/kaggle/input/cafa6-embeds",

    # --- Input Labels & Metadata (Dataset c95-cafa6) ---
    "LABEL_DIR": "/kaggle/input/c95-cafa6",
    "VOCAB_FILE": "vocab_C95_remove.csv",
    "TARGET_FILE": "train_targets_C95.pkl",
    "TRAIN_IDS": "train_ids_C95_split.npy",
    "VAL_IDS": "val_ids_C95_split.npy",
    "IA_FILE": "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv",

    # --- Model Params ---
    "input_dim": 1280,
    "hidden_dims": [2048, 4096],
    "dropout": 0.3,
    "batch_size": 32,
    "lr": 3e-4,
    "weight_decay": 1e-4,
    "epochs": 20,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

# ============================================================================

# 2. DATASET 

# ============================================================================


class CAFA6Dataset(Dataset):

    def __init__(self, ids_file, targets_file, embed_dir, num_classes):

        self.ids = np.load(ids_file)

        with open(targets_file, "rb") as f:

            self.labels_dict = pickle.load(f)

        self.num_classes = num_classes

        # Mapping ID -> Index 

        self.id_to_embed_idx = {}

        with open(os.path.join(embed_dir, "train_ids.txt"), "r") as f:

            for idx, line in enumerate(f):

                self.id_to_embed_idx[line.strip()] = idx

        # Mmap embedding

        self.embed_matrix = np.load(os.path.join(embed_dir,
                                                 "train_embeds.npy"),
                                    mmap_mode="r")

    def __len__(self):

        return len(self.ids)

    def __getitem__(self, idx):

        prot_id = self.ids[idx]

        embed_idx = self.id_to_embed_idx.get(prot_id)

        # Load X

        if embed_idx is None:

            feat = torch.zeros(1280, dtype=torch.float)

        else:

            feat = torch.from_numpy(
                self.embed_matrix[embed_idx].copy()).float()

        # Load Y

        target = torch.zeros(self.num_classes, dtype=torch.float)

        indices = self.labels_dict.get(prot_id, [])

        if len(indices) > 0:

            target[indices] = 1.0

        return feat, target


# ============================================================================

# 3. MODEL: WIDE MLP

# ============================================================================


class WideProteinMLP(nn.Module):

    def __init__(self,
                 input_dim,
                 num_classes,
                 hidden_dims=[2048, 4096],
                 dropout=0.3):

        super().__init__()

        layers = []

        layers.append(nn.LayerNorm(input_dim))

        prev = input_dim

        for h in hidden_dims:

            layers.append(nn.Linear(prev, h))

            layers.append(nn.GELU())

            layers.append(nn.Dropout(dropout))

            prev = h

        layers.append(nn.Linear(prev, num_classes))

        self.net = nn.Sequential(*layers)

    def forward(self, x):

        return self.net(x)


# ============================================================================

# 4. LOSS: ASL OPTIMIZED 

# ============================================================================


class AsymmetricLossOptimized(nn.Module):

    def __init__(
        self,
        gamma_neg=4,
        gamma_pos=0,
        clip=0.05,
        eps=1e-8,
        disable_torch_grad_focal_loss=True,
    ):

        super(AsymmetricLossOptimized, self).__init__()

        self.gamma_neg = gamma_neg

        self.gamma_pos = gamma_pos

        self.clip = clip

        self.eps = eps

        # Tắt gradient ở phần tính weight để tiết kiệm bộ nhớ

        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss

    def forward(self, x, y):

        # x: logits

        # y: labels (0/1)

        # 1. Tính xác suất P

        x_sigmoid = torch.sigmoid(x)

        xs_pos = x_sigmoid

        xs_neg = 1 - x_sigmoid

        # 2. Asymmetric Clipping (Shift xác suất âm)

        if self.clip is not None and self.clip > 0:

            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # 3. Tính Pt (Xác suất của nhãn đúng)

        # pt = p nếu y=1, pt = p_shifted nếu y=0

        pt = y * xs_pos + (1 - y) * xs_neg

        # 4. Tính Log Pt (Base Cross Entropy)

        # log(pt + eps)

        log_pt = torch.log(pt.clamp(min=self.eps))

        # 5. Tính hệ số điều chỉnh (Modulating Factor)

        if self.disable_torch_grad_focal_loss:

            torch.set_grad_enabled(False)

        one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)

        mod_factor = (1 - pt)**one_sided_gamma

        if self.disable_torch_grad_focal_loss:

            torch.set_grad_enabled(True)

        # 6. Final Loss

        loss = -mod_factor * log_pt

        return loss.sum()


# ============================================================================

# 5. METRIC & TRAINING LOOP

# ============================================================================


def calculate_fmax_subset(preds, targets, ia_weights):
    w = ia_weights.reshape(1, -1)
    true_sum = np.sum(targets * w, axis=1)
    valid_mask = true_sum > 0
    if valid_mask.sum() == 0: return 0.0
    
    p_sub = preds[valid_mask]
    t_sub = targets[valid_mask]
    w_sub = w 
    w_true_sub = true_sum[valid_mask]
    
    best_f1 = 0.0
    # Quét 51 ngưỡng cho nhanh
    thresholds = np.linspace(0.0, 1.0, 51) 
    
    for tau in thresholds:
        cut = (p_sub >= tau).astype(int)
        tp = np.sum((cut * t_sub) * w_sub, axis=1)
        pred_sum = np.sum(cut * w_sub, axis=1)
        
        prec = np.divide(tp, pred_sum, out=np.zeros_like(tp), where=pred_sum!=0)
        rec = np.divide(tp, w_true_sub, out=np.zeros_like(tp), where=w_true_sub!=0)
        
        avg_p = np.mean(prec)
        avg_r = np.mean(rec)
        
        if (avg_p + avg_r) > 0:
            f1 = 2 * avg_p * avg_r / (avg_p + avg_r)
        else:
            f1 = 0.0
            
        if f1 > best_f1: best_f1 = f1
            
    return best_f1


def main():

    print(
        f"Loading vocab from {os.path.join(CONFIG['LABEL_DIR'], CONFIG['VOCAB_FILE'])}..."
    )

    vocab_df = pd.read_csv(
        os.path.join(CONFIG["LABEL_DIR"], CONFIG["VOCAB_FILE"]))

    num_classes = len(vocab_df)

    try:

        ia_df = pd.read_csv(CONFIG["IA_FILE"],
                            sep="\t",
                            names=["term", "ia"],
                            header=None)

        ia_dict = ia_df.set_index("term")["ia"].to_dict()

        ia_weights = np.array(
            [ia_dict.get(t, 1.0) for t in vocab_df["term"].values])

        print("✅ Loaded Real IA Weights.")

    except:

        print("⚠️ Warning: IA file not found. Using weights=1.0")

        ia_weights = np.ones(num_classes)

    # Datasets

    train_ds = CAFA6Dataset(
        os.path.join(CONFIG["LABEL_DIR"], CONFIG["TRAIN_IDS"]),
        os.path.join(CONFIG["LABEL_DIR"], CONFIG["TARGET_FILE"]),
        CONFIG["EMBED_DIR"],
        num_classes,
    )

    val_ds = CAFA6Dataset(
        os.path.join(CONFIG["LABEL_DIR"], CONFIG["VAL_IDS"]),
        os.path.join(CONFIG["LABEL_DIR"], CONFIG["TARGET_FILE"]),
        CONFIG["EMBED_DIR"],
        num_classes,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=CONFIG["batch_size"],
        shuffle=True,
        num_workers=2,
        pin_memory=True,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=CONFIG["batch_size"] * 2,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    # Model & Training

    model = WideProteinMLP(CONFIG["input_dim"], num_classes,
                           CONFIG["hidden_dims"],
                           CONFIG["dropout"]).to(CONFIG["device"])

    if torch.cuda.device_count() > 1:

        print("Using", torch.cuda.device_count(), "GPUs!")

        model = nn.DataParallel(model)

    model = model.to(CONFIG["device"])

    criterion = AsymmetricLossOptimized(gamma_neg=4, gamma_pos=0, clip=0.05)

    optimizer = optim.AdamW(model.parameters(),
                            lr=CONFIG["lr"],
                            weight_decay=CONFIG["weight_decay"])

    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=CONFIG["lr"],
        steps_per_epoch=len(train_loader),
        epochs=CONFIG["epochs"],
    )

    print(f"\n🚀 Start Training WideMLP ({num_classes} classes)...")

    best_score = 0.0

    for epoch in range(CONFIG["epochs"]):

        model.train()

        total_loss = 0

        pbar = tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False)

        for x, y in pbar:

            x, y = x.to(CONFIG["device"]), y.to(CONFIG["device"])

            optimizer.zero_grad()

            logits = model(x)

            loss = criterion(logits, y)

            loss.backward()

            optimizer.step()

            scheduler.step()

            total_loss += loss.item()

            pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        # Validation Phase

        model.eval()

        all_preds, all_targets = [], []

        with torch.no_grad():

            for x, y in val_loader:

                logits = model(x.to(CONFIG["device"]))

                all_preds.append(torch.sigmoid(logits).cpu().numpy())

                all_targets.append(y.numpy())

        Y_p = np.vstack(all_preds)

        Y_t = np.vstack(all_targets)

        scores = {}

        for aspect in ["MFO", "BPO", "CCO"]:

            cols = vocab_df.index[vocab_df["aspect"] == aspect].tolist()

            if cols:

                scores[aspect] = calculate_fmax_subset(
                    Y_p[:, cols], Y_t[:, cols], ia_weights[cols])

        avg_fmax = np.mean(list(scores.values()))

        print(
            f"Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f} | Val F-max={avg_fmax:.4f} {scores}"
        )

        if avg_fmax > best_score:

            best_score = avg_fmax

            torch.save(model.state_dict(), "best_model_wide.pth")

            print("   🏆 Saved Best Model!")
